In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


In [1]:
## Billy Wang
## Geog 458 
## 2/23/2019

# Overview Of Data
 Some of the issues associated with Census Data include microdata and statistical confidentiality, as well as interpolated values.
 The data that we are using relies heavily on population demographic data, which is notoriously inaccurate as it is typically calculated
 in the old fashioned sense with polls & surveys, and a lot of people fall through the cracks, namely non-residential citizens and migrant citizens.
 Secondly, seeing as the preimse of urbanization hinges on the relationship between population size to total land mass area, the accuracy of both
 must be retained before calculations for titles of "urban" or "rural" can begin. The U.S Census Bureau takes more than just populaiton size over total sq ft into consideration
 because many more factors play a role in determinign urbanization status, however we do not have access to those values.
 Much of the confusion surrounding identity has to do with the Census' conceptualization of race and ethnicity and the limited choices that are presented [(McKenney & Bennett 1994; Alba 2018)](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1402238/).
 Microdata are records of information on individual respondents. Since the Census Bureau releases a fair amount of data to the public, including microdata, this raises issues of privacy, confidentiality, and protection against disclosure of individuals [(Hawala 2003; McCaa & Ruggles 2002)](http://users.pop.umn.edu/~rmccaa/census_microdata_revolution.pdf)<br>